# Flask Playground

### This notebook containers Selenium Simulation Code that would preform the data capture from ecWIN.

In [276]:
import sys
import json
import datetime
import time

import pprint
pp = pprint.PrettyPrinter(indent=4)

from pymongo import MongoClient
from bson.objectid import ObjectId
from influxdb import InfluxDBClient
from influxdb import DataFrameClient


import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

import time
import datetime

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import get_close_matches
import random

## Configure Mongodb

In [2]:
mongoClient = MongoClient('localhost',27017)
db = mongoClient.witsECOMP

## Configure InfluxDB

In [218]:
databaseName = "timeseriesdata"
influxClient = DataFrameClient("localhost", 8086, "root", "root", database=databaseName)
influxClient.create_database(databaseName)

In [217]:
influxClient.drop_database(databaseName)

## Read in time Series from excel spreadsheet
Note: this process can take a long time (2 to 3 minutes!).

In [4]:
spreadsheet = pd.read_excel('/home/chris/Desktop/Wits Data/WITS ecWIN7 Data/WITS ecWIN7 Data 2018.xlsx',
                            sheet_name='WITS ecWIN7 Data 2018',
                            index_col="DateTime"
                           )

## Read in building names and geojson information
These files were manually created by generating geojson from http://geojson.io/ and identifying the corisponding building names.

In [83]:
buildingSensorNames = spreadsheet.columns
print(buildingSensorNames)

Index(['WITS 13 Jubilee Road_kVA', 'WITS 13 Jubilee Road_kVarh',
       'WITS 13 Jubilee Road_kWh', 'WITS 3 Jubilee Road_kVA',
       'WITS 3 Jubilee Road_kVarh', 'WITS 3 Jubilee Road_kWh',
       'WITS 5 Jubilee Road_kVA', 'WITS 5 Jubilee Road_kVarh',
       'WITS 5 Jubilee Road_kWh', 'WITS 9 Jubilee Road_kVA',
       ...
       'WITS WC Village Zesti Lemonz_kWh', 'WITS WC WITS Club GEN 100 kVa_kVA',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kVarh',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kWh', 'WITS WC Wits Club_kVA',
       'WITS WC Wits Club_kVarh', 'WITS WC Wits Club_kWh',
       'WITS Wits Club Summation_kVA', 'WITS Wits Club Summation_kVarh',
       'WITS Wits Club Summation_kWh'],
      dtype='object', length=986)


In [84]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'stroke': '#555555', 'stroke-width': 2, 'stroke-opacity': 1, 'fill': '#555555', 'fill-opacity': 0.5, 'buildingName': 'Chamber of Mines'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28.0270779132843, -26.19199336514455], [28.027067184448242, -26.19195485633902], [28.02739977836609, -26.19192116112372], [28.027292490005493, -26.191271322922137], [28.026670217514038, -26.19134352734578]]]}}, {'type': 'Feature', 'properties': {'stroke': '#555555', 'stroke-width': 2, 'stroke-opacity': 1, 'fill': '#555555', 'fill-opacity': 0.5, 'buildingName': 'Flower Hall'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.025929927825928, -26.19151200416024], [28.025962114334106, -26.191950042737417], [28.026278614997864, -26.191901906710605], [28.0261

In [219]:
for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    mostLiklySensor = get_close_matches(buildingName + "_kVA", buildingSensorNames)[0]
    dataFrame = spreadsheet[mostLiklySensor].to_frame()
    influxClient.write_points(dataFrame, databaseName,{'buildingNumber': index, 'buildingName': buildingName, 'sensorName': mostLiklySensor})
    print(buildingName + "->" + mostLiklySensor)

Chamber of Mines->WITS WC Chamber of Mines Total_kVA
Flower Hall->WITS WC Flower Hall_kVA
Ithemba Labs High Voltage->WITS Ithemba Labs High Voltage_kVA
Wits Science Stadium->WITS WC Science Staduim TRF 2_kVA
Mathematical Science Building->WITS WC Maths Science Building_kVA
Zesti Lemonz->WITS EC PDH Zesti Lemon_kVA
FNB Building->WITS WC FNB Building Total_kVA
PEC PIMD Office Block->WITS PEC PIMD Office Block_kVA
DJ Du Plesis->WITS WC DJ du Plessis Building_kVA
West Campus Village->WITS Campus Lodge_kVA
Convocation Dining Hall->WITS WC Convocation Dining Hall GEN 125 kVA_kVA
Barnato Residence->WITS WC Barnato Sub Residence A - D_kVA
David Webster Hall->WITS WC David Webster Hall_kVA
The Barns->WITS WC The Barns_kVA
Olives and Plates->WITS WBS Olives & Plates Meter 2_kVA
Johannesburg Planetarium->WITS EC Planetarium_kVA
Wartenweilier Library->WITS EC Wartenweiler Library_kVA
Origins Centre->WITS EC Origin Centre_kVA
Chemistry Building->WITS EC Physics Building_kVA
Physics Building->WITS E

## Preform Queries to load data into the mongoDB.

These queries MUST pull from the influxdb as this is where all the time series data is stored. As new records are added from Selenium (or directly from tnew sensors) they will be added to the Influx DB. this querying and updating mongo process occures every 30 minutes to generate "fresh" graphs.

First, we need to make the database store for each building

In [332]:
buildingInformationStore = db.buildingInformation
# buildingInformation_id = buildingInformationStore.insert_one(BuildingChartInformation).inserted_id

Then, itterate over all the buildings that have been identified in the geojson and query for values for that building from influxdb for the predefined length of time.

In [333]:
startTime = "2018-08-19T00:00:00+00:00"
plotsToDraw = {
    'Day':{
        "resampleLength":'1H',
        "windowLength": 2*24 #1 day window with 2 samples per hour
        },
    'Week':{
        "resampleLength":'1D',
        "windowLength":2*24*7 # 1 week window with 2 samples per hour
        },
    'Year':{
        "resampleLength":'1M',
        "windowLength":2*24*240 #1 year window with 2 samples per hour
        },
}

for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    
    print(buildingName)
    buildingBlob = {
        "buildingID": index,
        "buildingName": buildingName,
        "sensorName": "SENSOR",
        "chartInformation": {
            "DayInformation": {
                "LastDay": [],
                "AverageDay":[]
            },
            "WeekInformation": {
                "LastWeek": [],
                "AverageWeek":[]
            },
            "YearInformation": {          
                "LastYear": [],
                "AverageYear":[]
            }
        },
    }
    for plotType in plotsToDraw:
        print(plotsToDraw[plotType]['resampleLength'], plotsToDraw[plotType]['windowLength'])
        query = "SELECT * FROM timeseriesdata WHERE buildingName='{}' AND time< '{}'".format(buildingName, startTime)
        queryResults = influxClient.query(query)
        results = queryResults["timeseriesdata"][queryResults["timeseriesdata"].keys()[0]]
#         print(results.tail(plotsToDraw[plotType]['windowLength']))
        resampledResults = results.tail(plotsToDraw[plotType]['windowLength']).resample(plotsToDraw[plotType]['resampleLength'], label='right').sum()
#         print(resampledResults.to_dict())
        print("\n")
        resultsDict = resampledResults.to_dict()
        formattedDict = {}
        for result in resultsDict:
            formattedDict[str(result)[0:19]] = resultsDict[result]
        buildingBlob["chartInformation"][plotType+"Information"]["Last" + plotType] = formattedDict
        buildingBlob["chartInformation"][plotType+"Information"]["Average" + plotType]=list(resampledResults)

    print(buildingBlob)    
    buildingInformation_id = buildingInformationStore.insert_one(buildingBlob)
#     print (buildingInformation_id)
    

# dataFrame = pd.DataFrame.from_dict(resultsDict)
# print(dataFrame)

Chamber of Mines
1H 48


1D 336


1M 11520


{'buildingID': 0, 'buildingName': 'Chamber of Mines', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 145.17023441840848, '2018-08-18 02:00:00': 142.2692258762089, '2018-08-18 03:00:00': 137.0727357562059, '2018-08-18 04:00:00': 136.55271513839682, '2018-08-18 05:00:00': 133.15015538115819, '2018-08-18 06:00:00': 134.2050197953401, '2018-08-18 07:00:00': 138.76943992972662, '2018-08-18 08:00:00': 183.57798103309602, '2018-08-18 09:00:00': 197.9376369596955, '2018-08-18 10:00:00': 216.354603149402, '2018-08-18 11:00:00': 213.302910783575, '2018-08-18 12:00:00': 220.675301402591, '2018-08-18 13:00:00': 215.06393790736, '2018-08-18 14:00:00': 178.6469660672619, '2018-08-18 15:00:00': 176.0618312497865, '2018-08-18 16:00:00': 168.9473004688703, '2018-08-18 17:00:00': 173.4287448240393, '2018-08-18 18:00:00': 171.9102988910057, '2018-08-18 19:00:00': 173.7684152994556, '2018-08-18 20:00:00': 172.



1D 336


1M 11520


{'buildingID': 4, 'buildingName': 'Mathematical Science Building', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 196.8530687226865, '2018-08-18 02:00:00': 197.7831220629423, '2018-08-18 03:00:00': 193.94790523818568, '2018-08-18 04:00:00': 191.3815181834057, '2018-08-18 05:00:00': 190.1337647694505, '2018-08-18 06:00:00': 191.3705574108347, '2018-08-18 07:00:00': 188.528046723634, '2018-08-18 08:00:00': 185.61371853493858, '2018-08-18 09:00:00': 183.3742670193657, '2018-08-18 10:00:00': 186.56275294380612, '2018-08-18 11:00:00': 172.22542309484982, '2018-08-18 12:00:00': 183.74530130277168, '2018-08-18 13:00:00': 178.9147054847952, '2018-08-18 14:00:00': 191.0453385990179, '2018-08-18 15:00:00': 197.44452820113702, '2018-08-18 16:00:00': 200.324571324916, '2018-08-18 17:00:00': 201.608049996057, '2018-08-18 18:00:00': 203.2, '2018-08-18 19:00:00': 207.36, '2018-08-18 20:00:00': 209.92048780374398, '2018-08-18 21

KeyboardInterrupt: 

In [281]:
loadedCampusInfo = {}
with open('../../Assets/otherinformation/campusInfomation.json') as json_data:
    loadedCampusInfo = json.load(json_data)
    print(loadedCampusInfo)

{'name': 'Main campus', 'center': ['28.0240, -26.18701'], 'numberOfStudents': 25000, 'averageWeekChart': {}, 'piDistribution': {}, 'sunburstDistribution': {}, 'streamGraph': {}, 'totalSensors': 2000}


In [11]:
campusInfoStore = db.campusInfo
campusInfo_id = campusInfoStore.insert_one(loadedCampusInfo).inserted_id

NameError: name 'db' is not defined

In [7]:
list(campusInfoStore.find({}))

[{'_id': ObjectId('5bbb60adb2831017c6117dbe'),
  'name': 'Main campus',
  'center': ['28.0240, -26.18701'],
  'numberOfStudents': 25000,
  'averageWeekChart': {},
  'piDistribution': {},
  'sunburstDistribution': {},
  'streamGraph': {},
  'totalSensors': 2000}]

In [8]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'features': [{'geometry': {'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28.0270779132843, -26.19199336514455], [28.027067184448242, -26.19195485633902], [28.02739977836609, -26.19192116112372], [28.027292490005493, -26.191271322922137], [28.026670217514038, -26.19134352734578]]], 'type': 'Polygon'}, 'properties': {'id': '0', 'buildingName': 'NOT Chamber Of Mines', 'style': {'color': '#999', 'fillColor': '#E04E5C', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}}, 'type': 'Feature'}, {'geometry': {'coordinates': [[[28.025929927825928, -26.19151200416024], [28.025962114334106, -26.191950042737417], [28.026278614997864, -26.191901906710605], [28.026192784309387, -26.19148312243793], [28.025929927825928, -26.19151200416024]]], 'type': 'Polygon'}, 'properties': {'id': '1', 'buildingName': 'Chamber Of Mines', 'style': {'color': '#999', 'fillColor': '#

In [9]:
geojsonStore = db.geojson
geojson_id = geojsonStore.insert_one(loadedGeoJson).inserted_id

In [10]:
list(geojsonStore.find({}))

[{'_id': ObjectId('5bbb60adb2831017c6117dbf'),
  'features': [{'geometry': {'coordinates': [[[28.026670217514038,
        -26.19134352734578],
       [28.026729226112362, -26.191767125729683],
       [28.026643395423886, -26.191786380165073],
       [28.026697039604187, -26.19204150113358],
       [28.0270779132843, -26.19199336514455],
       [28.027067184448242, -26.19195485633902],
       [28.02739977836609, -26.19192116112372],
       [28.027292490005493, -26.191271322922137],
       [28.026670217514038, -26.19134352734578]]],
     'type': 'Polygon'},
    'properties': {'id': '0',
     'buildingName': 'NOT Chamber Of Mines',
     'style': {'color': '#999',
      'fillColor': '#E04E5C',
      'fillOpacity': 0.7,
      'opacity': 1,
      'weight': 2}},
    'type': 'Feature'},
   {'geometry': {'coordinates': [[[28.025929927825928, -26.19151200416024],
       [28.025962114334106, -26.191950042737417],
       [28.026278614997864, -26.191901906710605],
       [28.026192784309387, -26.19

In [11]:
BuildingChartInformation = {}
with open('../../Assets/ChartInformation/BuildingChartInformation.json') as json_data:
    BuildingChartInformation = json.load(json_data)
    print(BuildingChartInformation)

{'buildingId': '0', 'dayInformation': {'LastDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'AverageDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'CampusAverageDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'CampusAverageDayNormalized': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24]}, 'weekInformation': {'LastWeek': [20, 14, 25, 16, 18, 22, 19], 'AverageWeek': [19, 14, 22, 14, 16, 19, 15], 'CampusAverageWeek': [28, 22, 30, 28, 28, 31, 22], 'CampusAverageWeekNormalized': [25, 19, 25, 26, 26, 28, 18]}, 'yearInformation': {'LastYear': [20, 14, 25, 16, 18, 22, 19], 'AverageYear': [19, 14, 22, 14, 16, 19, 15], 'CampusAverageYear': [28, 22, 30, 28, 28, 31, 22], 'CampusAverageYearNormalized': [25, 19, 25, 26, 26, 28, 18]}, 'histogramInformation': {}}


In [12]:
buildingInformationStore = db.buildingInformation
buildingInformation_id = buildingInformationStore.insert_one(BuildingChartInformation).inserted_id

In [13]:
list(buildingInformationStore.find({"buildingId": "0"}))

[{'_id': ObjectId('5bbb60adb2831017c6117dc0'),
  'buildingId': '0',
  'dayInformation': {'LastDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'AverageDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'CampusAverageDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'CampusAverageDayNormalized': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24]},
  'weekInformation': {'LastWeek': [20, 14, 25, 16, 18, 22, 19],
  